In [4]:
from collections import namedtuple,deque
from heapq import heappush, heappop
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [5]:
PUZZLE_DIM = 5
action = namedtuple('Action', ['pos1', 'pos2'])

In [6]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [7]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[21, 19, 14,  9, 20],
       [ 4, 12, 11,  5, 24],
       [23,  7,  1, 13,  6],
       [17, 22,  3, 15,  8],
       [ 2, 18, 10, 16,  0]])

In [8]:
class CustomHeuristicService:
    def __init__(self, goal_state: np.ndarray, puzzle_dim: int):
        self.goal_state = goal_state
        self.puzzle_dim = puzzle_dim

    def heuristic_weighted_manhattan(self, state: np.ndarray) -> int:
        """
        Modified Manhattan distance with increased weight.
        """
        distance = 0
        size = state.shape[0]
        for row in range(size):
            for col in range(size):
                value = state[row][col]
                if value != 0:
                    target_row = (value - 1) // size
                    target_col = (value - 1) % size
                    distance += 2 * (abs(row - target_row) + abs(col - target_col))  # Weighted by 2
        return distance

    def heuristic_combined_conflict(self, state: np.ndarray) -> int:
        """
        Modified linear conflict combining row and column checks in a single pass.
        """
        size = state.shape[0]
        conflicts = 0
        for i in range(size):
            row_max, col_max = -1, -1
            for j in range(size):
                # Row conflict
                row_value = state[i][j]
                if row_value != 0 and (row_value - 1) // size == i:  # Correct row
                    if row_value > row_max:
                        row_max = row_value
                    else:
                        conflicts += 2
                # Column conflict
                col_value = state[j][i]
                if col_value != 0 and (col_value - 1) % size == i:  # Correct column
                    if col_value > col_max:
                        col_max = col_value
                    else:
                        conflicts += 2
        return conflicts

    def heuristic_adjusted_walking_distance(self, state: np.ndarray) -> int:
        """
        Modified walking distance with squared penalties for far tiles.
        """
        size = state.shape[0]
        distance_grid = [[0] * size for _ in range(size)]
        for row in range(size):
            for col in range(size):
                value = state[row][col]
                if value != 0:
                    target_row = (value - 1) // size
                    target_col = (value - 1) % size
                    distance_grid[row][col] = (
                        abs(row - target_row) ** 2 + abs(col - target_col) ** 2
                    )
        return sum(sum(row) for row in distance_grid)

    def heuristic_misplaced_tiles(self, state: np.ndarray) -> int:
        """
        New heuristic: Count the number of misplaced tiles.
        """
        size = state.shape[0]
        misplaced = 0
        for row in range(size):
            for col in range(size):
                value = state[row][col]
                if value != 0:
                    goal_row = (value - 1) // size
                    goal_col = (value - 1) % size
                    if row != goal_row or col != goal_col:
                        misplaced += 1
        return misplaced

    def compute_multiplication_factor(self) -> int:
        """
        Dynamically adjusts the heuristic multiplier based on the puzzle dimension.
        """
        if self.puzzle_dim <= 4:
            return 1
        elif self.puzzle_dim == 5:
            return 5
        elif 5 < self.puzzle_dim <= 6:
            return 10_000
        else:
            return int(5_000 * (self.puzzle_dim ** 1.2))  # Scaled for larger puzzles

    def combined_heuristic(self, state: np.ndarray) -> int:
        """
        Combines the modified heuristics using a weighted sum.
        """
        factor = self.compute_multiplication_factor()
        return factor * (
            1.5 * self.heuristic_weighted_manhattan(state)  # Weighted Manhattan
            + 1.2 * self.heuristic_combined_conflict(state)  # Modified linear conflict
            + 0.8 * self.heuristic_adjusted_walking_distance(state)  # Adjusted walking
            + 0.5 * self.heuristic_misplaced_tiles(state)  # Misplaced tiles
        )


In [9]:
def a_star_solver_with_custom_heuristics(start_state: np.ndarray, goal_state: np.ndarray) -> tuple[list, int]:
    """
    Solves the n^2-1 puzzle using A* search with custom heuristics.
    """
    puzzle_dim = start_state.shape[0]
    heuristic_service = CustomHeuristicService(goal_state, puzzle_dim)

    open_list = []
    visited = set()

    start_hashable = tuple(map(tuple, start_state))
    goal_hashable = tuple(map(tuple, goal_state))

    heappush(open_list, (0, 0, start_hashable, []))  # (f, g, state_tuple, path)

    while open_list:
        f, g, current_hashable, path = heappop(open_list)

        if current_hashable == goal_hashable:
            solution_path = [np.array(state) for state in path + [current_hashable]]
            return solution_path, len(visited)

        visited.add(current_hashable)

        current_state = np.array(current_hashable)
        for move in available_actions(current_state):
            next_state = do_action(current_state, move)
            next_hashable = tuple(map(tuple, next_state))

            if next_hashable in visited:
                continue

            h = heuristic_service.combined_heuristic(next_state)
            new_g = g + 1
            new_f = new_g + h

            heappush(open_list, (new_f, new_g, next_hashable, path + [current_hashable]))

    return None, len(visited)


In [10]:
# The goal state
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

# Randomized start state (already provided in your code)
print("Randomized Start State:")
print(state)  # Use the `state` already generated in your code
print("\nGoal State:")
print(goal_state)

# Solve the puzzle using the A* solver with overestimation
solution_path, states_evaluated =  a_star_solver_with_custom_heuristics(state, goal_state)

# Print the results
if solution_path:
    print("\nSolution Found!")
    print(f"Number of Moves: {len(solution_path) - 1}")
    print(f"States Evaluated: {states_evaluated}")
    print("\nReplay Solution Steps:")
      # Print each state in the solution path
    for idx, step in enumerate(solution_path):
        print(f"\nStep {idx}:")
        print(step)

else:
    print("\nNo Solution Found!")


Randomized Start State:
[[21 19 14  9 20]
 [ 4 12 11  5 24]
 [23  7  1 13  6]
 [17 22  3 15  8]
 [ 2 18 10 16  0]]

Goal State:
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24  0]]

Solution Found!
Number of Moves: 200
States Evaluated: 3666

Replay Solution Steps:

Step 0:
[[21 19 14  9 20]
 [ 4 12 11  5 24]
 [23  7  1 13  6]
 [17 22  3 15  8]
 [ 2 18 10 16  0]]

Step 1:
[[21 19 14  9 20]
 [ 4 12 11  5 24]
 [23  7  1 13  6]
 [17 22  3 15  0]
 [ 2 18 10 16  8]]

Step 2:
[[21 19 14  9 20]
 [ 4 12 11  5 24]
 [23  7  1 13  6]
 [17 22  3  0 15]
 [ 2 18 10 16  8]]

Step 3:
[[21 19 14  9 20]
 [ 4 12 11  5 24]
 [23  7  1 13  6]
 [17 22  3 16 15]
 [ 2 18 10  0  8]]

Step 4:
[[21 19 14  9 20]
 [ 4 12 11  5 24]
 [23  7  1 13  6]
 [17 22  3 16 15]
 [ 2 18  0 10  8]]

Step 5:
[[21 19 14  9 20]
 [ 4 12 11  5 24]
 [23  7  1 13  6]
 [17 22  3 16 15]
 [ 2  0 18 10  8]]

Step 6:
[[21 19 14  9 20]
 [ 4 12 11  5 24]
 [23  7  1 13  6]
 [17  0  3 16 15]
 [ 2 22 18 10  